In [0]:
import bs4
import requests
import pandas as pd
import os
data_path = "./drive/My Drive/cs472/data/data.pkl"
data_path2 = "./drive/My Drive/cs472/data/data2.pkl"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

"""
Scrape Data from


https://www.imdb.com/search/title/?title_type=movie&view=advanced&genres=comedy&start=0

loop through <div> class="lister-item mode-advanced" </div>:

    title in class="lister-item-header"

    Rating: class="certificate"

    Length: class="runtime"

    Rating: class="inline-block ratings-imdb-rating" attirbute, data-value="8.6"

    Description class="text-muted"

    Director and actors: in label "class" every other is a name

"""


def scrape_data(filename):
    """
    gets data from imdb

    saves as csv
    :return:
    """

    if os.path.exists(filename):
        df = pd.read_pickle(filename)
        s = df.iloc[-1].genre + 1
    else:
        df = pd.DataFrame(columns=["title","s_rating","runtime","genre","description","director","actor","rating_label"])
        s = 0

    genres = ["comedy","sci-fi", "horror","romance","action","thriller","drama","mystery","crime",
              "animation","adventure","fantasy","comedy-romance","action-comedy","superhero"]

    ratings_dict = dict()

    for i,r in enumerate(["Unrated", "Not Rated", "NC-17", "R", "PG-13", "PG", "G"]):
        ratings_dict[r] = i

    genre_dict = dict()
    for i,g in enumerate(genres):
        genre_dict[g] = i

    get_url = lambda g, start: "https://www.imdb.com/search/title/?title_type=movie&view=advanced&sort=alpha,asc&genres="+g+"&start="+start

    for ge in genres:
        genre = genre_dict[ge]
        print(ge, genre)
        for s in range(100000,150000,51):
            url = get_url(ge, str(s))
            response = requests.get(url)
            if response.status_code == 200:
                html_doc = response.text
                soup = bs4.BeautifulSoup(html_doc, 'html.parser')
                items = soup.find_all("div", {"class": "lister-item mode-advanced"})
                for item in items:

                    rating_label = item.find("div", {"class", "inline-block ratings-imdb-rating"})

                    if rating_label == None:
                        continue
                    else:
                        rating_label = rating_label['data-value']

                    title = item.find("h3", {"class": "lister-item-header"}).find('a').text.replace("#","").replace("\"","")
                    
                    rating_value = item.find("span", {"class", "certificate"})
                    if rating_value is None:
                        continue
                    if rating_value.text not in ratings_dict.keys():
                        rating_value = len(ratings_dict)
                    else:
                        rating_value = ratings_dict[rating_value.text]                        

                    run_time = item.find("span", {"class", "runtime"})
                    if run_time == None:
                        continue

                    run_time = int(run_time.text.replace(",","").split()[0])

                    description = item.find_all("p", {"class", "text-muted"})
                    if description != None:
                        description = description[-1].text.strip()


                    ghost = item.find_all("span", {"class","ghost"})
                    if len(ghost) == 0:
                        continue
                    people = item.find_all("span", {"class","ghost"})[-1].parent.find_all('a')

                    if len(people) == 0:
                        continue
                    else:
                        director = people[0].text
                        actor = people[1].text


                    new_row = [title, rating_value, run_time, genre, description, director, actor, rating_label]
                    df.loc[len(df)] = new_row

            else:
                raise ConnectionError("Request not Successful:"+response.status_code)
        df.to_pickle(filename)

    return df
